# BCDP CORDEX Example
Here we will do a quick walkthrough of using BCDP to process some regional climate simulations from the Coordinate Regional Downscaling Experiment

In [ ]:
import os
import glob
import numpy as np
import bcdp

Create a file conventions template. Here we only need the model and variable names so the rest of the file template is filled in by wildcards.

In [ ]:
project = 'CORDEX-Africa'
template = '*_{model}_*_{variable}.nc'
bcdp.build_extractor(project, template, name_field='model', index=[1, 7])

Load the data. Because we have loaded the template, the loader now knows exactly how to extract the required informations from the filenames.

In [ ]:
paths = os.path.join(os.path.expanduser('~'), 'CORDEX-Africa_data/*clt*')
ens = bcdp.load_local(paths=paths, project=project)
print(ens.size)

The loader returns an `Ensemble` object, which is essentially a collection of datasets and applies preprocessing operations to each of them. Here we will regrid the data to a coarser (0.88 degree) grid using bilinear interpolation, and consider only the winter months (DJF).

In [ ]:
output_grid = bcdp.utils.grid_from_res((0.88, 0.88), ens.overlap)
ens = ens.homogenize(freq='Y', season='DJF', backend='scipy', method='linear',
                     output_grid=output_grid, clean=False)

Now that the underlying datastructures are homogeneous (same grid and time step), we can convert it to an xarray dataarray which has dimensions (names, time, lat, lon).

In [ ]:
da = ens.bundle('CORDEX').add_mean('CORDEX').first

We can easily visualize the annual climatology with xarray's built-in plotting methods.

In [ ]:
da.mean('time').plot(x='x', y='y', col='names', col_wrap=3)